# ETL Project

### Group Project by:

##### Bay Rodriguez  /  Chris Tran  /  Emil Rodulfo  /  Tim Besaw

In [ ]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine
import sys

sys.path.append("static/assets/Resources/")
import config

## Extract Data

In [ ]:
# Load and read game file
game_file = 'static/assets/Resources/game_stats.csv'
game_df = pd.read_csv(game_file, encoding='utf-8')

# Minimize data to bowl games from 2007-2016 and filter down fields pertinent to analysis
new_game_df = game_df.loc[game_df['Year'] >2006, :]
new_game_df = new_game_df[['Year', 'Away Team', 'Home Team', 'Bowl', 'id', 'Home Bowl Scores', 'Away Bowl Scores', 'Winning Team', 'Losing Team']]

new_game_df.head()

In [ ]:
# Load and read player file
player_file = 'static/assets/Resources/player_stats.csv'
player_df = pd.read_csv(player_file, encoding='utf-8')

# Filter data to fields pertinent to analysis
new_player_df=player_df[['GameId','team','player']]

new_player_df.head()

## Transform Data

In [ ]:
# Rename game dataframe columns to match database field names
xform_game = new_game_df.rename(columns={'Year': 'year', 'Away Team': 'away_team', 'Home Team': 'home_team', 'Bowl': 'bowl', 'id': 'gameid', 'Home Bowl Scores': 'home_bowl_scores', 'Away Bowl Scores': 'away_bowl_scores', 'Winning Team': 'winning_team', 'Losing Team': 'losing_team'})

# Set index
xform_game.set_index('gameid', inplace=True)

xform_game.head()

In [ ]:
# Rename player dataframe columns to match database field names
xform_player = new_player_df.rename(columns={'GameId':'gameid', 'team':'team', 'player':'player'})

# Set index
xform_player.set_index('gameid', inplace=True)

xform_player.head()

## Load Data

In [ ]:
# Create database connection
conn = (f'{config.username}:{config.password}@localhost/football_db')

engine = create_engine(f'mysql://{conn}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
# Load game dataframe to SQL database
xform_game.to_sql(name='game_stats', con=engine, if_exists = 'append', index=True)

In [ ]:
# Load players dataframe to SQL database
xform_player.to_sql(name='player_stats', con=engine, if_exists = 'append', index=True)